In [ ]:
from collections import deque

# Функція для пошуку збільшуючого шляху (BFS)
def bfs(capacity_matrix, flow_matrix, source, sink, parent):
    visited = [False] * len(capacity_matrix)
    queue = deque([source])
    visited[source] = True

    while queue:
        current_node = queue.popleft()
        
        for neighbor in range(len(capacity_matrix)):
            # Перевірка, чи є залишкова пропускна здатність у каналі
            if not visited[neighbor] and capacity_matrix[current_node][neighbor] - flow_matrix[current_node][neighbor] > 0:
                parent[neighbor] = current_node
                visited[neighbor] = True
                if neighbor == sink:
                    return True
                queue.append(neighbor)
    
    return False

# Основна функція для обчислення максимального потоку
def edmonds_karp(capacity_matrix, source, sink):
    num_nodes = len(capacity_matrix)
    flow_matrix = [[0] * num_nodes for _ in range(num_nodes)]  # Ініціалізуємо матрицю потоку нулем
    parent = [-1] * num_nodes
    max_flow = 0

    # Поки є збільшуючий шлях, додаємо потік
    while bfs(capacity_matrix, flow_matrix, source, sink, parent):
        # Знаходимо мінімальну пропускну здатність уздовж знайденого шляху (вузьке місце)
        path_flow = float('Inf')
        current_node = sink

        while current_node != source:
            previous_node = parent[current_node]
            path_flow = min(path_flow, capacity_matrix[previous_node][current_node] - flow_matrix[previous_node][current_node])
            current_node = previous_node

        # Оновлюємо потік уздовж шляху, враховуючи зворотний потік
        current_node = sink
        while current_node != source:
            previous_node = parent[current_node]
            flow_matrix[previous_node][current_node] += path_flow
            flow_matrix[current_node][previous_node] -= path_flow
            current_node = previous_node

        # Збільшуємо максимальний потік
        max_flow += path_flow

    return max_flow, flow_matrix

# ---------------- BUILD GRAPH -----------------
n = 22  # 20 вершин + штучний sink + штучне джерело (index 21)
capacity_matrix = [[0] * n for _ in range(n)]

# Terminals -> Warehouses
capacity_matrix[0][2] = 25  # T1 -> S1
capacity_matrix[0][3] = 20  # T1 -> S2
capacity_matrix[0][4] = 15  # T1 -> S3

capacity_matrix[1][4] = 15  # T2 -> S3
capacity_matrix[1][5] = 30  # T2 -> S4
capacity_matrix[1][3] = 10  # T2 -> S2

# Warehouses -> Stores
capacity_matrix[2][6] = 15  # S1 -> M1
capacity_matrix[2][7] = 10  # S1 -> M2
capacity_matrix[2][8] = 20  # S1 -> M3

capacity_matrix[3][9]  = 15  # S2 -> M4
capacity_matrix[3][10] = 10  # S2 -> M5
capacity_matrix[3][11] = 25  # S2 -> M6

capacity_matrix[4][12] = 20  # S3 -> M7
capacity_matrix[4][13] = 15  # S3 -> M8
capacity_matrix[4][14] = 10  # S3 -> M9

capacity_matrix[5][15] = 20  # S4 -> M10
capacity_matrix[5][16] = 10  # S4 -> M11
capacity_matrix[5][17] = 15  # S4 -> M12
capacity_matrix[5][18] = 5   # S4 -> M13
capacity_matrix[5][19] = 10  # S4 -> M14

# Підключаємо всі магазини до штучного sink
for store in range(6, 20):
    capacity_matrix[store][20] = float("inf")

# Artificial super-source (index 21) -> Terminals
capacity_matrix[21][0] = float("inf")  # super-source -> T1
capacity_matrix[21][1] = float("inf")  # super-source -> T2

# -----------------------------------------------

# ---------------- CALCULATE MAX FLOW -----------------
max_flow, flow_matrix = edmonds_karp(capacity_matrix, 21, 20)  # super-source -> sink

print(f"Максимальний потік у логістичній мережі = {max_flow}")


# ---------------- COMPUTE FLOW FROM TERMINALS TO STORES -----------------
def compute_terminal_flows(flow_matrix, terminals, stores):
    flow_table = {t: {s: 0 for s in stores} for t in terminals}

    for t in terminals:
        queue = [(t, float("inf"))]  # start with terminal and infinite flow
        while queue:
            node, flow_so_far = queue.pop(0)  # BFS style
            for neighbor in range(len(flow_matrix)):
                if flow_matrix[node][neighbor] > 0:
                    min_flow = min(flow_so_far, flow_matrix[node][neighbor])
                    if neighbor in stores:
                        flow_table[t][neighbor] += min_flow
                    queue.append((neighbor, min_flow))
                    flow_matrix[node][neighbor] -= min_flow

    return flow_table


terminals = [0, 1]
stores = list(range(6, 20))
flow_table = compute_terminal_flows(flow_matrix, terminals, stores)

# ---------------- PRINT RESULTS -----------------
print(f"{'Термінал':<10} {'Магазин':<10} {'Фактичний Потік':<15}")
for t in flow_table:
    for s in flow_table[t]:
        print(f"{'T'+str(t+1):<10} {'M'+str(s-5):<10} {flow_table[t][s]:<15}")

Максимальний потік у логістичній мережі = 115
Термінал   Магазин    Фактичний Потік
T1         M1         15             
T1         M2         10             
T1         M3         0              
T1         M4         15             
T1         M5         10             
T1         M6         5              
T1         M7         15             
T1         M8         10             
T1         M9         0              
T1         M10        0              
T1         M11        0              
T1         M12        0              
T1         M13        0              
T1         M14        0              
T2         M1         0              
T2         M2         0              
T2         M3         0              
T2         M4         0              
T2         M5         0              
T2         M6         0              
T2         M7         5              
T2         M8         0              
T2         M9         0              
T2         M10        20             
T2  